<a href="https://colab.research.google.com/github/Ads369/Ads_2s/blob/main/ipynb/Lesson%2025/25_4_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Навигация по уроку**

1. [Первое знакомство с AutoML](https://colab.research.google.com/drive/1bCWyzlp1-tcvt7TE60m4hFnRd5AWscWY)
2. [Гиперпараметры и оптимизация моделей](https://colab.research.google.com/drive/1CN69NftfVXUliyv11FGbM7qOYbO0XON5)
3. [AutoML в Keras](https://colab.research.google.com/drive/1V7mfY8da0S-FbWxhQbchJM38JSJBmtoZ)
4. Домашняя работа

В домашней работе необходимо с помощью AutoKeras или KerasTuner найти оптимальную модель для решения одной из следующей задач:

1. На 3 балла. Обучите модель с точностью не менее 90% предсказывать сарказм в новостных заголовках. Составьте 5 произвольных заголовков, которых нет в датасете и проверьте на них обученную модель, сделайте выводы. Ссылка на [датасет](https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip).
2. На 4 балла. Используйте [русский корпус новостей от Lenta.ru](https://www.kaggle.com/datasets/yutkin/corpus-of-russian-news-articles-from-lenta/data) подберите и обучите модель классифицировать новости по заголовкам на классы (поле topic в датасете). Используйте 9 самых часто встречаемых топиков и 10-й для остальных, не вошедших в 9 классов. Оцените модель с помощью отчета о классификации, сделайте выводы.  
3. На 5 баллов. Найдите публичный датасет по обращениям граждан в администрацию, техническую поддержку или за консультацией. Обучите модель классифицировать обращения по тематикам. Сформируйте отчет о классификации и матрицу ошибок.

In [1]:
!wget https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip
!unzip -qo "Sarcasm_Headlines_Dataset_v2.json.zip" -d ./dataset

--2025-01-21 07:56:19--  https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1789636 (1.7M) [application/x-zip-compressed]
Saving to: ‘Sarcasm_Headlines_Dataset_v2.json.zip’

Sarcasm_Headlines_D 100%[===================>]   1.71M  1.44MB/s    in 1.2s    

2025-01-21 07:56:21 (1.44 MB/s) - ‘Sarcasm_Headlines_Dataset_v2.json.zip’ saved [1789636/1789636]



In [2]:
!pip install tensorflow==2.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.0
    Uninstalling wrapt-1.17.0:
      Successfully uninstalled wrapt-1.17.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard
    Found existing installation

In [3]:
!pip install autokeras==1.1.0 tensorflow==2.15.1 keras-nlp==0.5.1

INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.1/527.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.9 MB/s eta 0:00:00
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.17.0
    Uninstalling tf_keras-2.17.0:
      Successfully uninstalled tf_keras-2.17.0


In [4]:
import tensorflow as tf
tf.__version__

'2.15.1'

In [5]:
import keras_nlp as nlp
nlp.__version__

'0.5.1'

In [6]:
# Библиотека матричного вычисления
import numpy as np
# Библиотека для работы с данными
import pandas as pd
# Библиотека для работы с регулярными выражениями
import re
# Библиотека для работы с фреймворком TensorFlow
import tensorflow as tf
# Библиотека AutoML autokeras
import autokeras as ak
# Библиотеки для построения графиков и их стилизации
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Утилита для расщепления выборки
from sklearn.model_selection import train_test_split

# Необходимые метрики для построения Матрицы ошибок и отчета о классификации
from sklearn.metrics import classification_report, confusion_matrix


import tensorflow as tf
import keras_nlp as nlp
import numpy as np
import pandas as pd
import re
import autokeras as ak
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.src.utils import to_categorical

In [7]:
address = "./dataset/Sarcasm_Headlines_Dataset_v2.json"
json_df = pd.read_json(address, lines = True ) # библиотека pandas умеет работать с json данными
df_sarcasm = pd.DataFrame(json_df) # создаем датафрейм

df_sarcasm.head() # выводим первые 5 записей датафрейма

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [8]:
# Ссылки на сами статьи нам не нужны
df_sarcasm = df_sarcasm.drop("article_link", axis = 1)

In [9]:
print('Найдено дубликатов: ', df_sarcasm.duplicated().sum())

# Удаляем дубликаты
df_sarcasm.drop_duplicates(subset=['headline'], inplace = True)

print('Осталось дубликатов после очистки: ', df_sarcasm.duplicated().sum())

Найдено дубликатов:  116
Осталось дубликатов после очистки:  0


In [10]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = " ".join(text.split())
    return text


df_sarcasm["headline"] = df_sarcasm["headline"].apply(preprocess_text)
df_sarcasm.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep totally nails why congress is falling ...
2,0,eat your veggies deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word stream...


In [21]:
X = (np.array(df_sarcasm.headline),)
y = (np.array(df_sarcasm.is_sarcastic),)

target = df_sarcasm.is_sarcastic
from keras.src.utils import to_categorical
y = to_categorical(target)


X_train, X_tmp, y_train, y_tmp = train_test_split(
    np.array(df_sarcasm.headline),
    np.array(df_sarcasm.is_sarcastic),
    test_size=0.2,
)

X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5)

In [22]:
print('Форма входных данных: ', X_train.shape)
print('Форма выходных меток: ', y_train.shape)
print('Пример заголовка: ', X_train[0])

Форма входных данных:  (22802,)
Форма выходных меток:  (22802,)
Пример заголовка:  paul ryan sitting among undecided voters at town hall debate


In [23]:
def check_class_balance(y):
    unique, counts = np.unique(y, return_counts=True)
    balance_dict = dict(zip(unique, counts))
    print("Class distribution:")
    for label, count in balance_dict.items():
        print(f"Class {label}: {count} samples ({count/len(y)*100:.2f}%)")
    return balance_dict


print("\nTraining set class balance:")
train_balance = check_class_balance(y_train)

print("\nValidation set class balance:")
val_balance = check_class_balance(y_val)

print("\nTest set class balance:")
test_balance = check_class_balance(y_test)


Training set class balance:
Class distribution:
Class 0: 11970 samples (52.50%)
Class 1: 10832 samples (47.50%)

Validation set class balance:
Class distribution:
Class 0: 1492 samples (52.35%)
Class 1: 1358 samples (47.65%)

Test set class balance:
Class distribution:
Class 0: 1489 samples (52.23%)
Class 1: 1362 samples (47.77%)


In [24]:
# Для экономии ОЗУ удаляем уже ненужные данные, после чего python запускает сборщик "мусора" для очистки памяти
del df_sarcasm, X_tmp, y_tmp

In [25]:
# Создание ансамбля моделей (3 наиболее подходящие к данным моделей)
clf = ak.TextClassifier(overwrite=True, max_trials=2, objective='val_accuracy')

# Обучаем 2 модели на 4-х эпохах, размер пакета подбирается автоматически
result_training = clf.fit(X_train, y_train, epochs=4, validation_data=(X_val, y_val))


Trial 2 Complete [00h 02m 12s]
val_accuracy: 0.8540350794792175

Best val_accuracy So Far: 0.8540350794792175
Total elapsed time: 00h 13m 49s
Epoch 1/4
713/713 [==============================] - 49s 62ms/step - loss: 0.4927 - accuracy: 0.7500 - val_loss: 0.3365 - val_accuracy: 0.8526
Epoch 2/4
713/713 [==============================] - 44s 62ms/step - loss: 0.2322 - accuracy: 0.9054 - val_loss: 0.5135 - val_accuracy: 0.8137
Epoch 3/4
713/713 [==============================] - 47s 67ms/step - loss: 0.1273 - accuracy: 0.9525 - val_loss: 0.4617 - val_accuracy: 0.8579
Epoch 4/4
713/713 [==============================] - 44s 62ms/step - loss: 0.0733 - accuracy: 0.9742 - val_loss: 0.5685 - val_accuracy: 0.8540


In [32]:
# Save the model
model = clf.export_model()
model.save("sarcasm_detection_model")

# Save hyperparameters
best_hyperparameters = clf.tuner.get_best_hyperparameters()[0]
with open("best_hyperparameters.txt", "w") as f:
  f.write(str(best_hyperparameters.values))

In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 expand_last_dim (ExpandLas  (None, 1)                    0         ['input_1[0][0]']             
 tDim)                                                                                            
                                                                                                  
 text_vectorization (TextVe  (None, 128)                  0         ['expand_last_dim[0][0]']     
 ctorization)                                                                                     
                                                                                              

In [28]:
import tensorflow as tf
import keras_nlp as nlp
import autokeras as ak

# Load the saved model
loaded_model = tf.keras.models.load_model("sarcasm_detection_model", custom_objects=ak.CUSTOM_OBJECTS)
# predictions = loaded_model.predict(new_data)